In [1]:
import jax
import projects.charformer.data as data
from importlib import reload
data = reload(data)
import sys
import numpy
import jax
import jax.numpy as jnp
numpy.set_printoptions(threshold=sys.maxsize)
# %pip install tqdm


2024-09-02 02:08:27.737269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 02:08:27.795342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 02:08:27.812912: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 02:08:29.591993: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [3]:
ds = data.CharDataset(data.CharDataset.get_default_config())

In [4]:
# !  wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt

In [5]:
# ds.create_packed_records(input_file_path='projects/charformer/data/TinyStoriesV2-GPT4-train.txt', output_dir='projects/charformer/data/tfrecords/', custom_delimiter='<|endoftext|>')

In [6]:
retokenized = ds.load_and_retokenize_tfrecord(file_path='projects/charformer/data/tfrecords/record_400.tfrecord')

In [7]:
for r in retokenized:
    print(r)


Once there was a pale piano. It was very shiny and looked like it had just been polished. Bob was a 3 year old boy who loved the piano. Every day, he would come and polish it. He always used his special color polishing cloth. Making the piano shiny and prettier was so much fun for him. 
One day, Bob was very excited. He had some new, sparkly polish that he wanted to use. He was so happy that he was dancing around the piano as he polished it. Suddenly, he knocked off one of the keys and it fell to the ground. Bob was so sad. He didn't know what to do. 
Luckily, his dad was there. He quickly fixed the piano and made it look as good as new. When he was finished, Bob was so proud. The piano was so pale and shiny. He gave his dad a big hug and thanked him for making the piano look so beautiful. From that day on, Bob always remembered to be very careful when polishing the piano.
<|endoftext|


Bella and Max are two cats who live in a big house with a nice family. They like to play together,

In [8]:
iter = ds.create_iterator('projects/charformer/data/tfrecords/record_*.tfrecord', batch_size=8)

In [9]:
batch = next(iter)

In [30]:
import minformer.model as model
import functools
model = reload(model)
data = reload(data)

In [31]:
cfg = model.Config(
    d_model=1024,
    ffw_multiplier=4,
    query_heads=8,
    key_heads=8,
    num_layers=4,
    key_dim=128,
    vocab_size=256,
    max_seq_len=8192,
    causal=True,
    use_attn_kernel=True,
    rules=model.fsdp_rules,
    mesh=model.create_mesh()
)

inference_config = model.Config(
    d_model=1024,
    ffw_multiplier=4,
    query_heads=8,
    key_heads=8,
    num_layers=4,
    key_dim=128,
    vocab_size=256,
    max_seq_len=8192,
    causal=True,
    use_attn_kernel=False,
    rules=model.mdl_parallel_rules,
    mesh=model.create_mesh()
)

In [32]:
weights = model.Weights.init(cfg, jax.random.PRNGKey(0), cfg.mesh, model.fsdp_rules)

In [ ]:
# jax.tree.map(lambda w: print(w.shape, w.dtype, jnp.mean(w)), weights)

In [33]:
step = jax.jit(model.update_step, static_argnames='cfg')
step = functools.partial(step, cfg=cfg)

In [50]:
loss, weights = step(weights, batch['x'], batch['segment_ids'], batch['y'])

In [56]:
test_batch = jnp.arange(0, 65)[None, :]
test_batch = jnp.repeat(test_batch, repeats = 8, axis=0)
{
    'x': test_batch[:-1],
    'y': test_batch[1:],
    'segment_ids': jnp.ones((8, 64)),
}

In [35]:
# Does loss go down!
for i in range(0, 1000):
    # batch = next(iter)
    loss, weights = step(weights, batch['x'], batch['segment_ids'], batch['y'])
    print(loss)

10.083326
9.970391
9.863377
9.771708
9.684777
9.602008
9.524451
9.448865
9.371531
9.298349
9.228609
9.156842
9.089424
9.017433
8.954483
8.898302
8.827738
8.76631
8.702787
8.645181
8.585813
8.535715
8.46744
8.40992
8.352292
8.302406
8.245361
8.193779
8.130066
8.073913
8.021753
7.9776783
7.929328
7.883355
7.835779
7.787507
7.741883
7.703205
7.6595445
7.622489
7.582061
7.543042
7.5092983
7.4854364
7.4704485
7.431034
7.4244566
7.349186
7.3162503
7.294123
7.281129
7.269762
7.266227
7.2068233
7.174762
7.1475916
7.155198
7.105288
7.0957365
7.078783
7.0944805
7.0330424
7.022174
7.0035367
6.9917507
6.990157
6.9951754
6.946828
6.9560995
6.925441
6.924077
6.897968
6.897412
6.84975
6.8411508
6.8293014
6.8454485
6.7890177
6.793463
6.777377
6.7591033
6.752569
6.7274036
6.7106905
6.7071886
6.68876
6.679841
6.6737614
6.6738267
6.689385
6.6152782
6.597597
6.595222
6.5919237
6.6027427
6.6123066
6.584985
6.550069
6.5498133
6.522031
6.5559416
6.491213
6.486394
6.495328
6.4740534
6.4656878
6.4609666
6.4493

KeyboardInterrupt: 

In [36]:
jax.tree.map(jnp.mean, weights)

Weights(layers=[Layer(q=Array(-0.000564575, dtype=bfloat16), k=Array(-0.000526428, dtype=bfloat16), v=Array(-0.00056076, dtype=bfloat16), proj=Array(-0.000518799, dtype=bfloat16), w1=Array(-0.00034523, dtype=bfloat16), w2=Array(-0.000358582, dtype=bfloat16), gamma1=Array(1, dtype=bfloat16), gamma2=Array(1, dtype=bfloat16)), Layer(q=Array(-0.000486374, dtype=bfloat16), k=Array(-0.000518799, dtype=bfloat16), v=Array(-0.000499725, dtype=bfloat16), proj=Array(-0.00056076, dtype=bfloat16), w1=Array(-0.000341415, dtype=bfloat16), w2=Array(-0.00034523, dtype=bfloat16), gamma1=Array(1, dtype=bfloat16), gamma2=Array(1, dtype=bfloat16)), Layer(q=Array(-0.000545502, dtype=bfloat16), k=Array(-0.000511169, dtype=bfloat16), v=Array(-0.000556946, dtype=bfloat16), proj=Array(-0.00056076, dtype=bfloat16), w1=Array(-0.000364304, dtype=bfloat16), w2=Array(-0.000366211, dtype=bfloat16), gamma1=Array(1, dtype=bfloat16), gamma2=Array(1, dtype=bfloat16)), Layer(q=Array(-0.000556946, dtype=bfloat16), k=Array(

In [351]:
model = reload(model)

In [43]:
cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)

In [51]:
ds.detokenize(batch['x'][0, :])

'Once upon a time, there was a little cat named Tim. Tim loved to eat tuna. One day, Tim saw a high shelf in the kitchen. He thought, maybe there is tuna up there.\nTim decided to climb up to the high shelf. He jumped and jumped until he reached the top. On the high shelf, he found a big can of tuna. Tim was very happy.\nTim ate all the tuna on the high shelf. His tummy was full and he was very happy. He climbed down and took a long nap. From that day on, Tim always looked for tuna on high shelves.\n<|endoftext|>One day, a brave cat named Tim went on a walk. He saw a big tree and decided to climb it. When he got to the top, he saw a pretty bird. The bird was sad because her egg lay on the ground.\n"Can you help me?" asked the bird. "My egg fell from my nest, and I cannot fly down to get it."\nTim, being a brave cat, said, "I will help you!" He climbed down the tree very carefully. He picked up the egg with his soft paws and climbed back up. He put the egg back in the bird\'s nest.\nThe

In [45]:
prompt = ds.tokenize('Once upon a time, there was a little cat named Tim.')
prompt, prompt_segment_ids = model.prepare_chunk(prompt, pad_to=256)

In [46]:
logits, cache = model.forward(prompt, prompt_segment_ids, weights, inference_config, cache)

In [47]:
sampled_tokens = model.sample_next_token(logits)

In [48]:
sampled_tokens

Array([[ 43,  39,  76,   9,  77,  72, 130,  17, 151,   4,  74, 162,  13,
        110, 187, 200,  47, 183, 135,  35,  55,   1,  38,  64, 174,  26,
         12, 223, 172,   3,   7, 185, 244, 137, 178, 165,  84,  14,   4,
          8,  53,  95,  46,  68, 123,  40,  67,  95,  20,  80, 228,  97,
        132,  97, 121, 115,  37, 144, 202,  40, 122,  46,  59,  94, 162,
        140, 142,  29,  41, 199,   1,  42, 147, 112,  65,  24,  20, 177,
        208,   7,  77, 255, 100,  92, 133,  50,  22,  40,  10,  55, 107,
          2,  35,  49, 108, 176, 146, 157, 196,  23,  15,   8,  83,   9,
        168, 253, 185,  23,  21,  71, 108,  19,  95,  83,  95,  95,  39,
         35,   3, 207, 133,  67, 142, 164, 189,  69,  25,  34, 229,  90,
         91, 228,  23, 119, 147,  83,  33,  19,  29,  99,  54, 157, 116,
        121, 111,  19, 122,  31,  37, 247,  41,  93,  60, 100, 120,  90,
         51,  16, 195,   6, 159,  39,  59, 253, 171,  19, 135,  19, 120,
          8,  43, 105, 133, 123,  29, 242,  91, 171

In [49]:
ds.detokenize(sampled_tokens[0, :])

'GC.8/*<oob>g<oob>3,<oob>c<oob><oob><oob>K<oob><oob>yS0B"<oob>pb<oob><oob>26<oob><oob><oob><oob><oob>@d37Q J&<oob>D% j<<oob>\n<oob>\n<oob><oob>A<oob><oob>D<oob>JW~<oob><oob><oob>sE<oob>0F<oob><oob>#nj<oob><oob>6/<oob>\x0c|<oob>NlD9Sñ1yMà<oob><oob><oob><oob>me7?8<oob><oob><oob>mk)ài ?  Cy2<oob><oob>%<oob><oob><oob>\'ox<oob>`{<oob>m<oob><oob>?wis\x0bR<oob><oob><oob><oob>i<oob>uA<oob>E}X\x0c<oob>`Of<oob>5<oob>CW<oob><oob>i<oob>i<oob>7G…<oob><oob>s<oob>{<oob><oob><oob><oob>%l<oob>J=<oob>u"<oob><oob>pñy$<oob>rna1<oob>=%<oob>´<oob>f"6.iv<oob><oob><oob><oob><oob>(<oob>}<oob><oob>a}A/<oob>p<oob><oob>q\\C<oob>EñF8<oob>}<oob><oob>O7v<unk>pq<oob>)<oob>é<oob><oob>z…'

In [405]:
ds.detokenize(batch['x'][0, :])

'One day, a nice girl named Lucy went to school. She wore a uniform. It was a pretty dress with a blue bow.\nAt school, Lucy saw her friend Tom. Tom had a hole in his uniform. "Oh no, Tom! Your uniform has a hole," she said. Tom was sad. He did not know what to do.\nLucy had an idea. She went home and asked her mom to help. They found a new uniform for Tom. They replaced the old one with the new one. Tom was happy. "Thank you, Lucy!" he said. They played together and had a great day at school.\n<|endoftext|>Once upon a time, in a small town, there was a little girl named Lucy. She loved to eat pastry. One day, she saw a pastry shop and went inside to buy her favorite treat. The shop was full of yummy pastries, and Lucy was very happy.\nLucy picked a pastry that was very hard. She didn\'t know why it was hard, but she wanted to try it. She took a bite, and suddenly, something unexpected happened. The hard pastry turned soft and sweet in her mouth. Lucy was very surprised and happy.\nThe

In [360]:
''.join([ds.itos[int(token)] for token in sampled_tokens[0, 1:3]])

'bv'

In [306]:
chunk_a = jnp.array([1, 2, 3, 4, 5, 6])
chunk_b = jnp.array([7, 8, 9])
chunk_c = jnp.array([1, 2, 3, 4, 5, 6, 7, 8, 9])




chunk_a, segment_ids_a = prepare_chunk(chunk_a, pad_to=16)
chunk_b, segment_ids_b = prepare_chunk(chunk_b, pad_to=16)
chunk_c, segment_ids_c = prepare_chunk(chunk_c, pad_to=16)

In [307]:
prefill_cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)
logits, prefill_cache = model.forward(chunk_c, segment_ids_c, weights, inference_config, prefill_cache)

In [308]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([9], dtype=int32),
 Array([0.111816, 0.0727539, 0.117188, 0.0407715, 0.12793, 0.0554199,
        0.0688477, 0.0678711, 0.120605, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=bfloat16),
 Array([[25, 25, 25, 25, 25, 67, 67, 67, 48, 48, 48, 48, 48, 48, 48, 48]],      dtype=int32))

In [309]:
prefill_cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)
logits, prefill_cache = model.forward(chunk_a, segment_ids_a, weights, inference_config, prefill_cache)

In [310]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([6], dtype=int32),
 Array([0.111816, 0.0727539, 0.117188, 0.0407715, 0.12793, 0.0554199, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0], dtype=bfloat16),
 Array([[25, 25, 25, 25, 25, 67, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48]],      dtype=int32))

In [311]:
logits, prefill_cache = model.forward(chunk_b, segment_ids_b, weights, inference_config, prefill_cache)

In [312]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([9], dtype=int32),
 Array([0.111816, 0.0727539, 0.117188, 0.0407715, 0.12793, 0.0554199,
        0.0688477, 0.0678711, 0.120117, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=bfloat16),
 Array([[67, 67, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48]],      dtype=int32))

AssertionError: K cache mismatch at layer 1

In [346]:
jnp.mean(logits_c, axis=-1)

Array([[1.2732564, 1.37221  , 1.4742929, 1.4115022, 1.225872 , 1.3535681,
        1.3480004, 1.2813404, 1.4418206, 1.1841507, 1.1841507, 1.1841507,
        1.1841507, 1.1841507, 1.1841507, 1.1841507]], dtype=float32)

In [345]:
jnp.mean(logits_b, axis=-1)

Array([[1.3480004, 1.2813404, 1.4418232, 1.1840515, 1.1840515, 1.1840515,
        1.1840515, 1.1840515, 1.1841507, 1.1841507, 1.1841507, 1.1841507,
        1.1841507, 1.1841507, 1.1841507, 1.1841507]], dtype=float32)

In [339]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([9], dtype=int32),
 Array([0.194336, 0.232422, 0.160156, 0.25, 0.283203, 0.235352, 0.228516,
        0.1875, 0.237305, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], dtype=bfloat16))

In [340]:
prefill_cache_c.lengths, jnp.mean(prefill_cache_c.k[3], axis=-1)[0, 0, :32]

(Array([9], dtype=int32),
 Array([0.194336, 0.232422, 0.160156, 0.25, 0.283203, 0.235352, 0.228516,
        0.1875, 0.237305, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], dtype=bfloat16))

In [145]:
q_segment_ids = jnp.array([[1,1,2,2], [1,1,1,2]])
k_segment_ids = jnp.array([[1,1,1,2,2,0,0], [1,1,2,2,2,0,0]])

In [151]:
(k_segment_ids[:, None, :] == q_segment_ids[:, :, None]).astype(jnp.int32)

Array([[[1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0]],

       [[1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0]]], dtype=int32)

In [156]:
jnp.tril(jnp.ones((3, 7)))

Array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0.]], dtype=float32)

In [16]:
jax.experimental.shard_map

<module 'jax.experimental.shard_map' from '/home/sholto/.local/lib/python3.10/site-packages/jax/experimental/shard_map.py'>

In [157]:
qk = (1, 1, 3, 7)
q_iota = jax.lax.broadcasted_iota(jnp.int32, qk, 2)
q_offset = 
k_iota = jax.lax.broadcasted_iota(jnp.int32, qk, 3)
mask = q_iota >= k_iota

In [162]:
q_iota.shape

(1, 1, 3, 7)

In [160]:
q_iota

Array([[[[0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2]]]], dtype=int32)